<a href="https://colab.research.google.com/github/junsung6140/Deep_leanring_tutorial/blob/main/Fashion_Mnist_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# 전체 6만개 데이터 중, 5만개는 학습 데이터용, 1만개는 테스트 데이터용으로 분리
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# image size는 28x28의 grayscale 2차원 데이터
print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)

## Mnist data 시각화

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(train_images[0], cmap='gray')
plt.title(train_labels[0])

In [ ]:
train_images[0, :, :], train_labels[0]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

def show_images(images, labels, ncols=8):
    figure, axs = plt.subplots(figsize=(22, 6), nrows=1, ncols=ncols) # 1행 ncols열 플랏을 만듬
    for i in range(ncols):
        axs[i].imshow(images[i], cmap='gray')
        axs[i].set_title(class_names[labels[i]])
        
show_images(train_images[:8], train_labels[:8], ncols=8)
show_images(train_images[8:16], train_labels[8:16], ncols=8)

## 데이터 전처리 수행
 * 0 ~ 255 사이의 픽셀값을 0~1 사이 값으로 변환 (보통 큰 값보다 0부터 1 사이 값이 결과가 잘나오는 경우가 많다)
 * array type은 float 32

In [ ]:
import numpy as np

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

def get_preprocessed_data(images, labels):
    
    # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변형. 
    images = np.array(images/255.0, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    
    return images, labels

train_images, train_labels = get_preprocessed_data(train_images, train_labels)
test_images, test_labels = get_preprocessed_data(test_images, test_labels)

print("train dataset shape:", train_images.shape, train_labels.shape)
print("test dataset shape:", test_images.shape, test_labels.shape)


In [ ]:
train_images[0]

## Dense Layer를 기반으로 모델을 생성

In [ ]:
INPUT_SIZE= train_images.shape[1:]

In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

model = Sequential([
    Flatten(input_shape=INPUT_SIZE), # 2차원 데이터를 dense로 학습하기 위해 flatten을 해준다 
    Dense(100, activation='relu'),
    Dense(30, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

### 모델의 Loss와 Optimizer 설정하고 학습을 수행
* loss는 categorical_corssentropy로, optimizer는 Adam으로 설정
* cotegorical crossentropy를 위해서 Lable을 One Hot Encoding으로 변경

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.utils import to_categorical

train_oh_labels = to_categorical(train_labels)
test_oh_labels = to_categorical(test_labels)

print(train_oh_labels.shape, test_oh_labels.shape)

In [ ]:
train_images.shape

In [ ]:
history = model.fit(x=train_images, y=train_oh_labels, batch_size=32, epochs=20, verbose=1)
## batch size는 32, 64 정도의 mini batch가 학습효과가 좋다고 알려져 있다. 
# batch size가 클수를 안정적으로 학습이 된다
# batch size가 작을 수록 필요한 메모리가 감소된다는 장점이 있다.

In [ ]:
print(history.history['loss'])
print(history.history['accuracy'])

## 테스트 data를 기반으로 Label 예측
- model.predcit()를 이용하여 label 예측
- predict() 인자로 입력되는 feature array는 학습의 feature array와 shape이 동일해야함
- fit() 시 3차원 array로 입력 햇으므로 predict()도 동일한 3차원 데이터 입력
- 특히 한건만 predict() 할때도 3차원 데이터 여야함. 이를 위해 expand_dims()로 2차원 데이터를 3차원으로 변경

In [ ]:
test_images.shape

In [ ]:
pred_proba = model.predict(test_images)
print(pred_proba.shape)

In [ ]:
pred_proba = model.predict(np.expand_dims(test_images[0], axis=0))
print('softmax output:', pred_proba)
pred = np.argmax(np.squeeze(pred_proba))
print('predicted class value:', pred)


In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
print('target class value:', test_labels[0], 'predicted class value:', pred)

## 테스트 데이터 세트로 모델 성능 검증

In [ ]:
model.evaluate(test_images, test_oh_labels, batch_size= 64)

## 검증 데이터 세트를 이용하여 학습 수행
- 일반적으로 fit()수행시 별도의 검증 데이텉 세트를 이용하여 학습 시 overfitting이 발생하는지 모니터링
- fit()을 수행하면 iteration을반복하기 때문에 중간에 파라미터 변경등의 작업이 어려움
- fit() iteration시 여러 작업을 하기 위해 callback객체를 가짐
- 검증 데이터 세트를 fit()시 적용하여 과적합이나 더이상 검증 데이터 성능이 좋아지지 않을 때 callback을 사용하여 learning rate 보정 작업 등을 수행 

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 기존 학습 데이터를 다시 학습과 검증 데이터 세트로 분리
tr_images, val_images, tr_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.15, random_state=2021)
print('train과 validation shape:', tr_images.shape, tr_labels.shape, val_images.shape, val_labels.shape)

# OHE 적용
tr_oh_labels = to_categorical(tr_labels)
val_oh_labels = to_categorical(val_labels)

print('after OHE:', tr_oh_labels.shape, val_oh_labels.shape)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

INPUT_SIZE = 28
model = Sequential([
    Flatten(input_shape=(INPUT_SIZE, INPUT_SIZE)),
    Dense(100, activation='relu'),
    Dense(30, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=128, validation_data=(val_images, val_oh_labels), 
                    epochs=20, verbose=1)

In [ ]:
print(history.history['loss'])
print(history.history['accuracy'])
print(history.history['val_loss'])
print(history.history['val_accuracy'])

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='valid')
plt.legend()

## Functional API
- 취미로 할거 아니면 꼭 알아둬야함
- Sequential을 이용하면 쉽게 모델을 만들 수 잇음
- 하지만 keras Framework의 핵심은 Functional API의 핵심이다

In [ ]:
# Sequential Model을 이용하여 Keras 모델 생성 
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential

INPUT_SIZE = 28

model = Sequential([
    Flatten(input_shape=(INPUT_SIZE, INPUT_SIZE)),
    Dense(100, activation='relu'),
    Dense(30, activation='relu'),
    Dense(10, activation='softmax')
])

model.summary()

In [ ]:
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
## 객체를 만들어준 후 생성자를 만들어 layer를 만듬

input_tensor = Input(shape=(INPUT_SIZE, INPUT_SIZE))
x = Flatten()(input_tensor)
x = Dense(100, activation='relu')(x)
x = Dense(30, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

model = Model(inputs=input_tensor, outputs=output)

model.summary()

## Custom 한 Dense Layer 생성하기

In [ ]:
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.models import Model
import tensorflow as tf

class CustomDense(tf.keras.layers.Layer):
    def __init__(self, units= 32):
      super(CustomDense, self).__init__()
      self.units= units

    def build(self, input_shape):
      self.w= self.add_weight(
          shape= (input_shape[-1], self.units),
          initializer= 'random_normal',
          trainable= True,
      )
      self.b= self.add_weight(
          shape= (self.units,), initializer= 'random_normal', trainable= True,
      )

      # CustomDense 객체에 callable로 입력된 입력 데이터 처리
    def call(self, inputs):
      return tf.matmul(inputs, self.w)+ self.b

  # input 값을 4개의 원소를 가지는 1차원 으로 생성
inputs= Input((4,))
  # 10개의 unit을 가지는 CustomDense 객체를 생성 후 callable로 inputs값 입력
outputs= CustomDense(10)(inputs)

  # inputs와 outputs로 model 생성
model= Model(inputs, outputs)
model.summary()

# 함수화

In [ ]:
from tensorflow.keras.layers import Layer, Input, Dense, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf

INPUT_SIZE= 28

def create_model():
  input_tensor = Input(shape= (INPUT_SIZE, INPUT_SIZE))
  x= Flatten()(input_tensor)
  x= Dense(100, activation= 'relu')(x)
  x= Dense(50, activation='relu')(x)
  output= Dense(10, activation= 'softmax')(x)

  model= Model(inputs= input_tensor, outputs= output)
  return model

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 0~1 사이값의 float32로 변경하는 함수
def get_preprocessed_data(images, labels):
  # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변경
  images= np.array(images/255.0, dtype=np.float32)
  labels= np.array(labels, dtype= np.float32)

  return images, labels

# 0~1 사이값 float32 변경하는 함수 호출 한 뒤 ohe 적용
def get_preprocessed_ohe(images, labels):
  images, labels= get_preprocessed_data(images, labels)
  # one hot encoding 적용
  oh_labels= to_categorical(labels)
  return images, oh_labels

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_labels = get_preprocessed_ohe(train_images, train_labels)
    test_images, test_oh_labels = get_preprocessed_ohe(test_images, test_labels)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    tr_images, val_images, tr_oh_labels, val_oh_labels = train_test_split(train_images, train_oh_labels, test_size=valid_size, random_state=random_state)
    
    return (tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels ) 

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
# Fashion MNIST 데이터 재 로딩 및 전처리 적용하여 학습/검증/데이터 세트 생성. 

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)
(tr_images, tr_oh_labels), (val_images, val_oh_labels), (test_images, test_oh_labels) = \
    get_train_valid_test_set(train_images, train_labels, test_images, test_labels, valid_size=0.15, random_state=2021)
print(tr_images.shape, tr_oh_labels.shape, val_images.shape, val_oh_labels.shape, test_images.shape, test_labels.shape)


In [ ]:
from tensorflow.keras.optimizers import Adam

# Model 생성 및 optimizer, loss, metric 적용
model = create_model()
model.summary()

model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# 학습 수행. 
history = model.fit(x=tr_images, y=tr_oh_labels, batch_size=128, epochs=20, validation_data=(val_images, val_oh_labels))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_history(history):
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='valid')
    plt.legend()
    
show_history(history)